In [14]:
from skimage.filters import threshold_local,threshold_otsu,try_all_threshold,threshold_mean,threshold_yen
import numpy as np
import cv2
from skimage.draw import rectangle
from skimage.morphology import skeletonize
from math import sqrt

In [3]:
def OrderPoints(pts):
    rect = np.zeros((4, 2), dtype = "float32")
    n=60
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]+n
    rect[2] = pts[np.argmax(s)]-n

    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]-n
    #x,y
    rect[1][0]-=n
    rect[1][1]+=n
    rect[3] = pts[np.argmax(diff)]
    #x,y
    rect[3][0]+=n
    rect[3][1]-=n
    return rect

def TransformPrespective(image, pts):
    rect = OrderPoints(pts)
    (tl, tr, br, bl) = rect
 
 
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
  [0, 0],
  [maxWidth - 1, 0],
  [maxWidth - 1, maxHeight - 1],
  [0, maxHeight - 1]], dtype = "float32")
 
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
 
    return warped

In [4]:
def GetMaxContour(img_dir,MORPH=9,CANNY=84,debug=False):
    """
    takes image directory ,
    structural rect dim(optional),canny max thres(optional)
    debug if true outputs images after each process
    returns a boolean specifying if the prespective shoud be adjusted,
    max contour approximation and grayoriginal image
    """
    img = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    img_copy = img.copy()
    hImg, wImg = img_copy.shape
    #smooth image
    cv2.GaussianBlur(img, (3,3), 0, img)
    # Remove small details and writing on paper
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(MORPH,MORPH))
    dilated = cv2.dilate(img, kernel)
    edges = cv2.Canny(dilated, 0, CANNY, apertureSize=3)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    #get edges again so we get inner line surrounding the paper
    #avoid paper outer edges being connected to noise in background
    edges = cv2.dilate(edges, kernel)
    edges = cv2.Canny(edges, 0, CANNY, apertureSize=3)
    # finding contours
    im,contours,heirarchy = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    maxArea=0
    adjustPrespective = False
    approx=[]
    # filter for max contour
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        if(w*h >= maxArea):
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.05 * peri, True)
            maxArea=w*h
            if(maxArea/(hImg*wImg) >= 0.1):
                adjustPrespective= True

    if(debug):
        orig = cv2.imread(img_dir)
        cv2.drawContours(orig,[approx], -1, (0, 255, 0), 1)
        cv2.imwrite("orig.png",img)
        cv2.imwrite("dilated.png",dilated)
        cv2.imwrite("edges.png",edges)
        cv2.imwrite('outline.png',orig)
    return adjustPrespective,approx,img_copy

In [5]:
def warpedPrespective(imgToWarp,approxContour,ratio=1,debug=False):
    warped = TransformPrespective(imgToWarp, approxContour.reshape(len(approxContour), 2) * ratio)
    if(debug):
        cv2.imwrite('result.png',warped)
    return warped

In [6]:
def RemoveShadow(img_gray,debug=False):
    #remove noise and writting on paper to leave out onlu the illumination
    dilated_img = cv2.dilate(img_gray, np.ones((7,7), np.uint8)) 
    blurred_img = cv2.medianBlur(dilated_img, 21)
    #remove any dark lighning aka shadow
    diff_img = 255 - cv2.absdiff(img_gray, blurred_img)
    norm_img = diff_img.copy()
    #normalization and word sharpening
    cv2.normalize(diff_img, norm_img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    _, thr_img = cv2.threshold(norm_img, 230, 0, cv2.THRESH_TRUNC)
    cv2.normalize(thr_img, thr_img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    if(debug):
        cv2.imwrite("shadowOut.png",norm_img)
        cv2.imwrite("thres.png",thr_img)
    return thr_img


In [7]:
def Binarize(img_gray,debug=False):
    thres =threshold_yen(img_gray)
    final = (img_gray >thres)*255
    if(debug):
        cv2.imwrite("final.png",final)
        #fig, ax = try_all_threshold(thr_img, figsize=(20, 30), verbose=False)
    return final


In [8]:
#remove lines function
def FloodFromCorners(im_th,debug=False):
    hImg, wImg = im_th.shape[:2]
    mask = np.zeros((hImg+2, wImg+2), np.uint8)
    im_floodfill = 255 - im_th
    ##################Flood fill corners#####
    cv2.floodFill(im_floodfill, mask, (0,0), 255)
    cv2.floodFill(im_floodfill, mask, (wImg-1,0), 255)
    cv2.floodFill(im_floodfill, mask, (0,hImg-1), 255)
    cv2.floodFill(im_floodfill, mask, (wImg-1,hImg-1), 255)
    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    if(debug):
        cv2.imwrite("Inverted_Floodfilled_Image.png", im_floodfill_inv)
        cv2.imwrite("Floodfilled_Image.png", im_floodfill)
    return im_floodfill_inv


######################cv contours#########
def getClosedShapes(im_filled,debug=False):
    hImg, wImg = im_filled.shape
    im2, contours_cv, hierarchy = cv2.findContours(im_filled, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    hierarchy = hierarchy[0]
    #[Next, Previous, First_Child, Parent]
    im_empty = np.ones((hImg, wImg,3), np.uint8) * 255
    i=0
    for cnt in contours_cv:
        #contours that has at least two children
        x,y,w,h = cv2.boundingRect(cnt)
        acceptedArea = h<=150 and w <=300 and h>10 and w>30
        child1Idx = hierarchy[i][2]
        hasChildren = child1Idx != -1 and  hierarchy[child1Idx][0] != -1
        if(hasChildren and acceptedArea):
            child2Idx = hierarchy[child1Idx][0]
        #this two children are close (letters)
        #check area
        if(hasChildren and acceptedArea):
            #print((w*h)/area)
            x1,y1,w1,h1 = cv2.boundingRect(contours_cv[child1Idx])
            x2,y2,w2,h2 = cv2.boundingRect(contours_cv[child2Idx])
            if(sqrt((x1-x2)**2+((y1-y2)**2)) <= 200):
                cv2.drawContours(im_empty, contours_cv, i, (0,0,0), 1)
        if(child1Idx != -1 and  hierarchy[child1Idx][0] == -1):
            x1,y1,w1,h1 = cv2.boundingRect(contours_cv[child1Idx])
            if(w1/w >= 0.3):
                cv2.drawContours(im_empty, contours_cv, i, (0,0,0), 1)
        if(not hasChildren and acceptedArea and w/h > 1.8 and w/h<=3.5):
            print(w/h,w,h)
            cv2.drawContours(im_empty, contours_cv, i, (0,0,0), 1)

        i+=1
    if(debug):
        cv2.imwrite("contoured.png", im_empty[:,:,0])
    return im_empty[:,:,0]

In [35]:
#integerated preprocessing
img_dir ="input/11.png"
adjustPrespective,approxContour,grayImg = GetMaxContour(img_dir)
warpedImg = grayImg
if(adjustPrespective):
    warpedImg = warpedPrespective(grayImg,approxContour)
shadowFreeImg = RemoveShadow(warpedImg,True)
binarizedImg = Binarize(shadowFreeImg,True)
filledImg = FloodFromCorners(binarizedImg.astype(np.uint8).copy(),True)
contourdImg = getClosedShapes(filledImg,True)
im2, contours, hierarchy = cv2.findContours(255 - contourdImg.astype(np.uint8).copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
image_result = binarizedImg.astype(np.uint8).copy()
for cnt in contours:
    x,y,w,h = cv2.boundingRect(cnt)
    rr, cc = rectangle(start = (y,x), end = (y+h,x+w), shape=contourdImg.shape)
    print(x,y,h,w)
    if(x==0 and y==0 ):
        continue
    n=10
    x = max(0,x-n)
    y = max(0,y-n)
    y2 = min(y+h+n,image_result.shape[0])
    x2 = min(x+w+n,image_result.shape[1])
    image_result[y:y2,x:x2] = 255
    
cv2.imwrite("image_result.png",image_result)

3.466666666666667 52 15
2.35 47 20
3.2142857142857144 90 28
2.0555555555555554 37 18
25 534 71 226
177 584 8 8
138 584 4 4
122 579 15 52
116 557 3 3
118 556 3 4
100 554 6 11
109 553 3 3
135 548 3 3
73 541 29 74
25 535 70 226
149 547 24 35
149 547 24 35
346 527 62 217
542 562 3 3
502 553 3 3
503 550 5 6
507 549 3 3
526 548 9 9
468 534 30 84
346 527 62 216
412 540 26 54
417 553 3 3
412 540 26 54
1207 506 5 8
1211 506 3 4
187 402 78 226
410 474 3 3
187 402 78 226
306 432 20 47
306 432 20 47
212 422 28 90
281 442 3 3
242 439 3 3
250 435 3 3
243 435 6 9
212 422 28 90
482 302 62 149
495 317 36 46
523 341 7 7
523 342 6 6
507 316 3 3
482 302 62 149
1059 287 61 284
1157 300 27 38
1233 288 3 4
1059 287 61 284
1119 311 18 37
1119 311 18 37
1200 306 32 34
1200 306 32 34
1278 303 18 19
1278 303 18 19
150 157 62 173
150 157 62 173
194 176 17 19
194 176 17 19
215 174 23 84
219 186 3 3
215 174 23 84
977 153 93 186
977 153 93 186
1027 176 23 83
1069 183 5 4
1027 176 23 83
353 35 51 189
354 35 51 188
41

True

In [92]:
#try to remove any opening in shapes
kernels = [cv2.getStructuringElement(cv2.MORPH_RECT,(1,4))]
kernels.append(np.array([
       [1,0, 0, 0],
       [0,1, 0, 0],
       [0,0, 1, 0],
       [0,0, 0, 1]], dtype=np.uint8))
kernels.append(np.array([
       [0, 0,0, 1],
       [0, 0, 1,0],
       [0, 1, 0,0],
       [1, 0, 0,0]], dtype=np.uint8))
kernels.append(cv2.getStructuringElement(cv2.MORPH_RECT,(4,1)))   
sk=((255-final)/255).astype(np.uint8)
sk = skeletonize(sk)
cv2.imwrite("sk.png",sk*255)
for kernel in kernels:
    opening=(255- sk*255).astype(np.uint8)
    opening = cv2.morphologyEx(opening, cv2.MORPH_OPEN, kernel,iterations=3)
    sk=((255-opening)/255).astype(np.uint8)
    sk = skeletonize(sk)
cv2.imwrite("opened.png",opening)
opening=(1 - opening/255).astype(np.uint8)
final_bin=(1 - final/255).astype(np.uint8)
cv2.imwrite("test1.png",opening*255)
cv2.imwrite("test2.png",final_bin*255)

connect = opening & final_bin
cv2.imwrite("connect.png",255 - connect*255)


NameError: name 'final' is not defined

In [24]:
dictImg = dict()
for i in range(len(warpedImg)):
    for j in range(len(warpedImg[0])):
        if warpedImg[i][j] not in dictImg:
            dictImg[warpedImg[i][j]] =0
        dictImg[warpedImg[i][j]]+=1
print(dictImg)

{255: 183940, 247: 76, 231: 59, 217: 64, 203: 53, 189: 64, 181: 33, 175: 56, 171: 49, 167: 72, 161: 57, 237: 66, 173: 79, 147: 52, 122: 14, 98: 71, 82: 28, 72: 34, 86: 44, 100: 74, 114: 53, 135: 53, 137: 53, 143: 22, 151: 62, 157: 67, 70: 61, 245: 78, 199: 51, 116: 68, 80: 24, 153: 47, 195: 23, 219: 29, 163: 32, 74: 44, 120: 34, 201: 51, 183: 44, 124: 45, 66: 59, 118: 27, 177: 51, 68: 49, 126: 426, 185: 37, 249: 73, 129: 487, 193: 44, 251: 65, 233: 64, 145: 41, 239: 42, 155: 33, 235: 52, 253: 90, 110: 46, 149: 59, 64: 57, 229: 38, 102: 32, 159: 44, 104: 51, 88: 33, 215: 40, 133: 38, 106: 35, 197: 17, 94: 41, 223: 19, 221: 49, 96: 34, 211: 52, 207: 31, 165: 42, 213: 34, 112: 48, 205: 70, 225: 29, 78: 39, 18: 61, 22: 60, 0: 1917, 209: 32, 131: 49, 14: 26, 243: 62, 92: 41, 60: 9, 46: 39, 24: 45, 139: 33, 141: 25, 187: 39, 54: 14, 8: 29, 6: 20, 169: 54, 227: 15, 50: 13, 12: 13, 90: 17, 62: 5, 20: 64, 241: 52, 2: 45, 52: 10, 40: 21, 16: 35, 42: 22, 28: 28, 179: 27, 38: 7, 191: 10, 44: 10, 1